In [ ]:
import mysql.connector as mysql

db = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "1234",
    database = "imagedescription"
)

In [ ]:
cursor = db.cursor()

cursor.execute("""
select id, unix_time, type, data from logger_traces lt
where type in ('keydown', 'keyup')
and user_id = 8
order by time;
""")

## 'fetchall()' method fetches all the rows from the last executed statement
events = cursor.fetchall()

downkeys = {}
prev_event = None
for event in events:
    (id, unix_time, type, key) = event
    
    # Use lowercase key because the key in the event will be upper or lowercase depending on whether shift is pressed.
    # The key value for the down event will be a capital if a capital is typed, but the up event may be lower case
    # if shift is released before the key is released.
    if key.__class__ == str:
        key = key.lower()
    
    if type == 'keydown':
        if not key in downkeys:
            downkeys[key] = event
        else:
            (id_down, unix_time_down, _, _) = downkeys[key]
            print(f"========= DOUBLE DOWNKEY EVENT: {event} PREVIOUS EVENT AT {unix_time_down}, ID {id_down}")            
            downkeys[key] = event
    else: # keyup
        if key in downkeys:
            (id_down, unix_time_down, _, _) = downkeys[key]
            DU = unix_time - unix_time_down
            if prev_event == None:
                DD = -1
                UD = -1
                UU = -1
            else:
                (prev_key, prev_unix_time_down, prev_unix_time_up) = prev_event
                DD = unix_time_down - prev_unix_time_down
                UD = unix_time_down - prev_unix_time_up
                UU = unix_time - prev_unix_time_up
            # print(f"key {key:10} down {unix_time_down} up {unix_time} DU {unix_time - unix_time_down} ids ({id_down}, {id})")
            print(f"key {prev_key:10} DU {DU:4} UD {UD:4} DD {DD:4} UU {UU:4} ids ({id_down:7}, {id:7})")
            prev_event = (key, unix_time_down, unix_time)
            downkeys.pop(key, None)
        else:
            print(f"========= UPKEY WITHOUT MATCHING DOWNKEY: {event}")

for key in downkeys:
    print(f"========= DOWNKEY EVENT WITHOUT MATCHING UPKEY: {downkeys[key]}")
    